# Buchbergers Algorithm

## Boiler Plate

First load the Sympy module

In [389]:
## Sympy Boiler Plate

from __future__ import division
from sympy import *
x, y, z, t = symbols('x y z t')
k, m, n = symbols('k m n', integer=True)
f, g, h = symbols('f g h', cls=Function)
init_printing()
init_printing(use_latex='mathjax', latex_mode='equation')


import pyperclip
def lx(expr):
    pyperclip.copy(latex(expr))
    print(expr)

import numpy as np
import matplotlib as plt

## Terminology

There are mutlidegree's, leading coefficients, leading monomials and leading terms.

In [390]:
def f(x, domain = 'QQ'):
    return 2+3*x+5*y+7*x*y+11*x**2*y+13*x*y**2+17*x**2*y**2+19*x**2*y**2*z**2
    # return 13*x**2+11*x-7
def g(x, domain = 'QQ'):
    return 2+3*x+5*y+7*x*y+11*x**2*y+13*x*y**2+17*x**2*y**2+19*x*y**2*z**2
    # return 13*x**2+11*x-7


In [391]:
g(x)

    2  2       2           2  2         2                        
17⋅x ⋅y  + 11⋅x ⋅y + 19⋅x⋅y ⋅z  + 13⋅x⋅y  + 7⋅x⋅y + 3⋅x + 5⋅y + 2

In [392]:
f(x)

    2  2  2       2  2       2           2                        
19⋅x ⋅y ⋅z  + 17⋅x ⋅y  + 11⋅x ⋅y + 13⋅x⋅y  + 7⋅x⋅y + 3⋅x + 5⋅y + 2

Notice how the order is lexical $\uparrow$

The **multidegree** is the largest power ocross all variables

In [393]:
def multideg(polynomial):
    return max(degree_list(polynomial))
multideg(f(x))

2

Occassionally this will be written as $\left\langle 2,2,2\right\rangle$

The *Leading Coefficient* is the coefficient corresponding to the Leading Monomial

In [394]:
LC(f(x))
# Can also be used as a method
# poly(f(x), domain = 'QQ').LC()

19

The leading monomial is the corresponding monomial

In [395]:
# Also known as the initial.
LM(f(x))

 2  2  2
x ⋅y ⋅z 

The leading term is the largest term, given the order

In [396]:
LT(f(x))

    2  2  2
19⋅x ⋅y ⋅z 

## S Polynomial

The $S$-Polynomial is a polynomial used in Buchberger's Algorithm, given some polynomials:

In [397]:
def f(x, y):
    return x**3*y**2-x**2*y**3+x
def g(x, y):
    return 3*x**4*y+y**2

f, g = f(x, y), g(x, y)    

The Lowest Common Multiple of the leading monomials can be given by:

In [398]:
from sympy.polys.monomials import monomial_mul, monomial_lcm, monomial_divides, term_div

In [399]:
# https://docs.sympy.org/latest/modules/polys/basics.html
LMf = LM(f)
LMg = LM(g)
# LCM12 = monomial_lcm(LMF, LMG) # This fails
LCM12 = lcm(LMf, LMg)
LCM12


 4  2
x ⋅y 

Then the $S$-polymial is given by:

In [400]:
def s_polynomial(f, g):
    LCM_fg = lcm(LM(f), LM(g))
    s = LCM_fg*(f/LT(f)-g/LT(g))
    return s.expand()
s=s_polynomial(f, g)
s

                3
   3  3    2   y 
- x ⋅y  + x  - ──
               3 

This can be used to determine if a polynomial belongs to a Groebner Bases ($G$) or not, this is known as Buchberger's Criterion. If the remainder of the S polynomial divided by all elements of $F$ is 0, the pair belongs in $G$, if not, the remainder belongs in $G$.

consider the set of functions:

In [401]:
F = [2*x-y, x**2-1-y]
F

⎡          2        ⎤
⎣2⋅x - y, x  - y - 1⎦

To determine if this is a Groebner Basis:

In [402]:
s=s_polynomial(F[0], F[1])
s

  x⋅y        
- ─── + y + 1
   2         

In [403]:
q,r = div(s,f*g)
r

  x⋅y        
- ─── + y + 1
   2         

This is non-zero and so this should be added back to $F$, however we should be more careful to calculate the remainder appropriately:

In [404]:
print(div(s, F[0]))
print(div(s, F[1]))

(-y/4, -y**2/4 + y + 1)
(0, -x*y/2 + y + 1)


This implies that:

$$
-\frac{xy}{2}+y+1 = -\frac{y}{4} (2x-y) + 0 (x^2-y-1) + (-\frac{y^2}{4} +y + 1 - \frac{xy}{2} + y + 1)
$$

So to calculate the remainder more robustly:

In [405]:
r_list = [ div(s, F[i])[1] for i in range(len(F)) ]
r = sum(r_list)
r

         2          
  x⋅y   y           
- ─── - ── + 2⋅y + 2
   2    4           

And adding that back in:

In [406]:
F.append(r)

Now the Remainder can be calculated again for $f_1$ and $f_3$

In [407]:
s=s_polynomial(F[0], F[2])
q,r = div(s,f*g)
r

   2          
- y  + 4⋅y + 4

As this is non-zero it should be added back in

In [408]:
F.append(r)
F

⎡                              2                          ⎤
⎢          2            x⋅y   y                2          ⎥
⎢2⋅x - y, x  - y - 1, - ─── - ── + 2⋅y + 2, - y  + 4⋅y + 4⎥
⎣                        2    4                           ⎦

Now comparing $f_1$ and $f_4$

In [409]:
s=s_polynomial(F[0], F[3])
q,r = div(s,f*g)
if r!=0:
    F.append(r)
    print(F)
else:
    print("This pair is a Groebner Basis")

[2*x - y, x**2 - y - 1, -x*y/2 - y**2/4 + 2*y + 2, -y**2 + 4*y + 4, 4*x*y + 4*x - y**3/2]


Now comparing $f_1$ and $f_5$

In [410]:
s=s_polynomial(F[0], F[4])
q,r = div(s,f*g)
r
if r!=0:
    F.append(r)
    print(F)
else:
    print("This pair is a Groebner Basis")

[2*x - y, x**2 - y - 1, -x*y/2 - y**2/4 + 2*y + 2, -y**2 + 4*y + 4, 4*x*y + 4*x - y**3/2, -x + y**3/8 - y**2/2]


Hmmm, this doesn't seem to be terminating, I'm going to have to put more research into this.

In [411]:
f = x**3-2*x*y
g = x**2*y+x-2*y**2
F = [f, g]

In [412]:
def poly_prod(F):
    product_val = 1
    for poly in F:
        product_val *= poly
    return product_val

In [413]:
def remainder_set_div(s, F):
    r_list = [ div(s, F[i])[1] for i in range(len(F)) ]
    r = sum(r_list)
    return r


In [414]:
def buchberger_criterion(f,g, F):
    s = s_polynomial(f, g)
    r = remainder_set_div(s, F)
    return r

In [415]:
r = buchberger_criterion(F[0], F[1], F)
if r != 0:
    F.append(r)


In [416]:
F

⎡ 3           2            2      2⎤
⎣x  - 2⋅x⋅y, x ⋅y + x - 2⋅y , -2⋅x ⎦

So we got back a remainder of $-x^2$, this isn't zero, so we throw it in the bag

We haven't tried every combination, so onto the next one:

In [417]:
r = buchberger_criterion(F[0], F[2], F)
if r != 0:
    F.append(r)


In [418]:
F

⎡ 3           2            2      2        ⎤
⎣x  - 2⋅x⋅y, x ⋅y + x - 2⋅y , -2⋅x , -6⋅x⋅y⎦

This gave a remainder of $-2xy$, so it goes in the bag

Now we try the next combination $f_1$ and $f_4$

In [419]:
r = buchberger_criterion(F[0], F[3], F)
if r != 0:
    F.append(r)
F

⎡ 3           2            2      2                2⎤
⎣x  - 2⋅x⋅y, x ⋅y + x - 2⋅y , -2⋅x , -6⋅x⋅y, -6⋅x⋅y ⎦

So the problem at this step is that diving by the product of F is not what we want

In [421]:
print(div(-2*x*y**2, F[1]))
print(div(-2*x*y**2, F[2]))
print(div(-2*x*y**2, F[3]))
print(div(-2*x*y**2, F[4]))

(0, -2*x*y**2)
(0, -2*x*y**2)
(y/3, 0)
(1/3, 0)
